In [19]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from PIL import Image
from PIL import ImageFilter as Filter

In [20]:
def edge_enhancing(array):
    method = np.random.choice(['ada_thold', 'laplacian', 'edge_enahnced'])
    
    if method=='ada_thold':     
        return np.expand_dims(cv2.adaptiveThreshold(array, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 1), 2)
    
    elif method=='laplacian':
        return np.expand_dims(cv2.Laplacian(array,cv2.CV_64F, ksize=5), 2)
    
    else:
        image = Image.fromarray(np.squeeze(array, axis=2)).convert('L')
        return np.expand_dims(np.asarray(image.filter(Filter.EDGE_ENHANCE_MORE)), 2)

In [21]:
def de_texturization(array):
    n = np.random.choice([5, 9, 13, 15])
    sigma = np.random.choice([50, 65, 75])
    
    return np.expand_dims(cv2.bilateralFilter(array, n, sigma, sigma), 2)

In [22]:
def random_flip(array):    
    if random.choice([True, False]):
        return tf.image.random_flip_left_right(array).numpy()
    else:
        return tf.image.random_flip_up_down(array).numpy()

In [23]:
def tumbnail(array, shape=(512,512)):
    return cv2.resize(array, shape) 

In [24]:
def random_crop(array):
    method = np.random.choice(['left', 'right', 'top', 'down'])
    v_center = array.shape[1]//2
    h_center = array.shape[0]//2
    
    if method == 'left':
        return array[:,:v_center,:]
    elif method == 'right':
        return array[:,v_center:,:]
    elif method == 'top':
        return array[:h_center,:,:]
    elif method == 'down':
        return array[h_center:,:,:]
    else:
        return array

In [25]:
def pre_procrssing(image):
    temp = np.asarray(image)
    
    if np.random.choice([True, False], p=[0.45, 0.55]):
        # other augmentaions
        temp = de_texturization(temp)
        
        # crop
        if random.choice([True, False]):
            temp = tf.image.random_crop(temp, (128,128,1)).numpy()
        else:
            temp = random_crop(temp)
        
        temp = tumbnail(temp, (input_size, input_size))

        return np.expand_dims(temp, 2)
    
    else:
        return temp

In [26]:
train_df = pd.read_csv('data/train_df.csv', usecols=['image_path', 'Target'])
train_df

,Target,image_path
0,0,./images/train/1.2.826.0.1.3680043.8.498.10025...
1,15,./images/train/1.2.826.0.1.3680043.8.498.10036...
2,12,./images/train/1.2.826.0.1.3680043.8.498.10038...
3,14,./images/train/1.2.826.0.1.3680043.8.498.10050...
4,3,./images/train/1.2.826.0.1.3680043.8.498.10053...
...,...,...
1733,14,./images/train/1.2.826.0.1.3680043.8.498.99711...
1734,11,./images/train/1.2.826.0.1.3680043.8.498.99727...
1735,3,./images/train/1.2.826.0.1.3680043.8.498.99935...
1736,2,./images/train/1.2.826.0.1.3680043.8.498.99969...


In [38]:
train_df['Target'] = train_df['Target'].str.strip()
train_df['image_path'] = train_df['image_path'].apply(lambda x: os.path.join("data", x.lstrip("./")))
train_df


,Target,image_path
0,0,data/data/images/train/1.2.826.0.1.3680043.8.4...
1,15,data/data/images/train/1.2.826.0.1.3680043.8.4...
2,12,data/data/images/train/1.2.826.0.1.3680043.8.4...
3,14,data/data/images/train/1.2.826.0.1.3680043.8.4...
4,3,data/data/images/train/1.2.826.0.1.3680043.8.4...
...,...,...
1733,10,data/data/images/train/1.2.826.0.1.3680043.8.4...
1734,13,data/data/images/train/1.2.826.0.1.3680043.8.4...
1735,13,data/data/images/train/1.2.826.0.1.3680043.8.4...
1736,0,data/data/images/train/1.2.826.0.1.3680043.8.4...


In [ ]:
from sklearn.model_selection import train_test_split

split_data = train_df[['image_path', 'Target']]
train, test = train_test_split(train_df[['image_path', 'Target']], test_size=0.2)
train.shape, test.shape


((1390, 2), (348, 2))

In [ ]:
class XrayDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame with columns "Target" and "image_path".
            transform (callable, optional): Transform to be applied on an image.
        """
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row["image_path"]
        # Convert label to an integer and then to a torch tensor (of type long)
        label = torch.tensor(int(row["Target"]), dtype=torch.long)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


# Define transformations (resize to 224x224 and convert to tensor)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create the dataset and dataloader
train_dataset = XrayDataset(train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = XrayDataset(test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes=16):
        """
        Args:
            num_classes (int): Number of target classes.
        """
        super(CNNClassifier, self).__init__()
        # Using padding to keep spatial dimensions, then reducing with pooling.
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Adaptive pooling to get a fixed feature size regardless of the input dimensions.
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [batch, 32, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))  # [batch, 64, 56, 56]
        x = self.avgpool(x)                   # [batch, 64, 7, 7]
        x = x.view(x.size(0), -1)             # flatten to [batch, 64*7*7]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)                     # logits output
        return x

# Determine the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# It's a good idea to automatically determine the number of classes.
num_classes = train_df["Target"].nunique()  
model = CNNClassifier(num_classes=num_classes).to(device)

# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 4: Training Loop
epochs = 10  # You can adjust the number of epochs as needed

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        # Ensure labels are the right type (long) for CrossEntropyLoss.
        labels = labels.to(device).long()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

# Optional: Save the trained model
torch.save(model.state_dict(), "xray_model.pth")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# First, accumulate predictions and true labels from the test dataset.
true_labels = []
pred_labels = []

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        # Ensure labels are tensors and in the correct type
        labels = torch.tensor(labels, device=device, dtype=torch.long)
        
        # Get model predictions
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        # Save predictions and true labels
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Compute the confusion matrix from the true and predicted labels.
conf_metric = confusion_matrix(true_labels, pred_labels)

# Normalize the confusion matrix by dividing each row by its sum (i.e., relative percentages).
conf_metric_normalized = conf_metric / np.sum(conf_metric, axis=1, keepdims=True)

# Plot the confusion matrix using Seaborn.
plt.figure(figsize=[12,12], dpi=100)
sns.heatmap(np.round(conf_metric_normalized, 2),
            cbar=False,
            annot=True,
            annot_kws={"size": 9},
            cmap=plt.cm.Blues)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.title('Normalized Confusion Matrix')
plt.show()